In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 1. Data Extraction: Using the BeautifulSoup

In [2]:
from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(res,'xml')
table = soup.find('table')

In [3]:
post=[]
borough=[]
neigh=[]
# loop by tag to extract
for tr in table.find_all('tr'):
    i=1
    p1=b1=n1=0
    b1_tag=n1_tag=''
    for td in tr.find_all('td'):
        if i == 1: 
            p1=td.text
        elif i == 2: 
            b1, b1_tag = td.text,td.find('a')
        elif i == 3: 
            n1,n1_tag=str(td.text).strip(),td.find('href')
        i +=1
    # uncomment the print to evaluate filtered out records
    try:
        if ((b1_tag is None) and (n1_tag is None)):
            print ('in try ==>','## b1 is:', b1,'## b1 tag is:', b1_tag,'## n1 is:', n1,'## n1 tag is:', n1_tag)
            continue
    except IndexError:
            continue
    if n1 =='Not assigned' and b1 != 'Not assigned':
        n1=b1
    if p1 != 0:
        post.append(p1), borough.append(b1), neigh.append(n1)

in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: York ## b1 tag is: None ## n1 is: Humewood-Cedarvale ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None

# 2. Data Transformation

In [4]:
dict = {'PostalCode':post, 'Borough':borough, 'Neighborhood':neigh}
df = pd.DataFrame.from_dict(dict)
df.to_csv('toronto.csv')

## Print the number of rows of your dataframe

In [5]:
df.shape

(209, 3)

#### Unfortunately geocoder didn't work. I used link to a csv file that has the geographical coordinates of each postal code and merged it with previous DataFrame to have proper output

In [6]:
csv_path = 'http://cocl.us/Geospatial_data'
geo = pd.read_csv(csv_path)

In [7]:
geo.columns = ['PostalCode', 'Latitude','Longitude']

In [8]:
geo_df = pd.merge(df, geo,how='outer', on='PostalCode')

#### First, I've tried to get coordinate of North York.

In [20]:
address = 'North York'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of North York are 43.7709163, -79.4124102.


### The output is correct, so we can proceed to use FOLIUM library!

### Here the aim is to generate maps, to visualize neighborhoods and see how they cluster together.

In [21]:
# create map using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_df['Latitude'], geo_df['Longitude'], geo_df['Borough'], geo_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

### Folium is a great visualization library. 

##### Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.